In [ ]:
import pandas as pd

path = "./data/main.csv"
df = pd.read_csv(path, dtype={'ЕДРПОУ':'object'})

df['Дата_госпіталізації'] = df['Дата_госпіталізації'].astype("datetime64[ns]")
df['Дата_виписки'] = df['Дата_виписки'].astype("datetime64[ns]")


df['Діагноз_основний'] = df.Діагноз_основний.fillna("")
df['Діагнози_додаткові'] = df.Діагнози_додаткові.fillna("")
df['Проведені_інтервенції'] = df.Проведені_інтервенції.fillna("")

## Розшифровка діагноза основного

In [ ]:
f = open("./data/diagnosis.csv", encoding='UTF-8')
d = dict()
for line in f:
    k = line.split('|')[0]
    v = line.split('|')[1]
    d[k] = v
    
    
df['Діагноз_основний'] = df['Діагноз_основний'].apply(lambda s: f"{s} - {d.setdefault(s, '')}") 

## Розшифровка діагноза основного та додаткових

In [ ]:
# словник береться з "Розшифровка діагноза основного"

def add_diagnos(s):
    if s == "" or s == "[]" or s == "{}":
        return ""
    
    
    if s[0] == "[":
        s = s[2:-2]
        lt = s.split("', '")
    else:
        s = s[1:-1]
        lt = s.split(",")
        
        
    st = ""
    for j, i in enumerate(lt):
        if j == len(lt) - 1:
            st = st + f"{i} - {d.setdefault(i, '')}"
        else:    
            st = st + f"{i} - {d.setdefault(i, '')} | "
            
            
    return st

df['Діагнози_додаткові'] = df['Діагнози_додаткові'].apply(add_diagnos)

## Розшифровка інтервенцій

In [ ]:
ff = open("./data/intervents.csv", encoding='UTF-8')
ff = [line.rstrip() for line in ff.readlines()]
dd = dict()
for ln in ff:
    k = ln.split('|')[0]
    v = ln.split('|')[1]
    dd[k] = v
    
def action(s):
    if s == "":
        return s
    
    
    if s[0] == "[":
        s = s[2:-2]
        lt = s.split("', '")
    else:
        s = s[1:-1]
        lt = s.split(",")
        
        
    st = ""
    for j, i in enumerate(lt):
        if j == len(lt) - 1:
            st = st + f"{i} - {dd.setdefault(i, '')}"
        else:    
            st = st + f"{i} - {dd.setdefault(i, '')} | "
            
            
    return st    

df['Проведені_інтервенції'] = df['Проведені_інтервенції'].apply(action)

## Розшифровка підстави госпіталізації

In [ ]:
fa = open("./data/admission_source.csv", encoding='UTF-8')
da = dict()
for lin in fa:
    k = lin.rstrip('\n').split(',')[0]
    v = lin.rstrip('\n').split(',')[1]
    da[k] = v

    
df['Підстава_госпіталізації'] = df['Підстава_госпіталізації'].fillna("")    
df['Підстава_госпіталізації'] = df['Підстава_госпіталізації'].apply(lambda s: s if s == "" else f"{da[s]}")

## Розшифровка результат_госпіталізації

In [ ]:
fdi = open("./data/discharge_mode.csv", encoding='UTF-8')
ddi = dict()
for li in fdi:
    k = li.rstrip('\n').split(',')[0]
    v = li.rstrip('\n').split(',')[1]
    ddi[k] = v

    
df['Результат_госпіталізації'] = df['Результат_госпіталізації'].fillna("")  
df['Результат_госпіталізації'] = df['Результат_госпіталізації'].apply(lambda s:  s if s == "" else f"{ddi[s]}")

## Розшифровка статі

In [ ]:
df['Стать'] = df['Стать'].apply(lambda s: "Чоловік" if s == "MALE" else "Жінка")

## Запис у файл result.xlsx

In [ ]:
df["Дата_госпіталізації"] = df["Дата_госпіталізації"].dt.date
df["Дата_виписки"] = df["Дата_виписки"].dt.date
df.to_excel("result.xlsx", index=False)
print("файл - result.xlsx записан")